In [1]:
%reload_ext autoreload
%autoreload 2

In [158]:
import os
import pandas as pd
from datetime import datetime,timedelta
from pytz import timezone
import requests as rq
from bs4 import BeautifulSoup as bs
from urllib.parse import urlencode, urljoin, urlparse
import time

from pathlib import Path
DATAPATH = Path(r'data')

In [3]:
os.chdir('..')

In [12]:
from mbp.webscraping import get_team_name_to_ids, activate_web_driver
from mbp.paths import RAW_DATA_DIR

In [5]:
driver = activate_web_driver("firefox")

Installing firefox...
executable_path: /Users/auser/.local/bin/geckodriver


In [6]:
team_ids = get_team_name_to_ids(driver, {})
team_ids

{'A&M-Corpus Christi': '560639',
 'Abilene Christian': '560649',
 'Air Force': '561195',
 'Akron': '560651',
 'Alabama': '560658',
 'Alabama A&M': '560654',
 'Alabama St.': '560655',
 'Alcorn': '560664',
 'American': '560666',
 'App State': '560667',
 'Arizona': '560672',
 'Arizona St.': '560669',
 'Ark.-Pine Bluff': '560622',
 'Arkansas': '560676',
 'Arkansas St.': '560674',
 'Army West Point': '561197',
 'Auburn': '560680',
 'Austin Peay': '560682',
 'BYU': '560702',
 'Ball St.': '560683',
 'Baylor': '560687',
 'Bellarmine': '561098',
 'Belmont': '560635',
 'Bethune-Cookman': '560689',
 'Binghamton': '560691',
 'Boise St.': '560693',
 'Boston College': '560695',
 'Boston U.': '560696',
 'Bowling Green': '560699',
 'Bradley': '560700',
 'Brown': '560704',
 'Bryant': '560705',
 'Bucknell': '560707',
 'Buffalo': '560709',
 'Butler': '560711',
 'CSU Bakersfield': '560714',
 'CSUN': '560723',
 'Cal Poly': '560712',
 'Cal St. Fullerton': '560718',
 'California': '560729',
 'California Bapt

In [138]:
df = pd.DataFrame.from_dict(team_ids.items())
df.head()

df = df.rename(columns={0: 'team_name', 1: 'team_id'})
df.reset_index()
# Save to raw data directory
df.to_csv(f"{RAW_DATA_DIR}/mbb_team_names_to_number.csv")

In [238]:
df = pd.read_csv(f"{RAW_DATA_DIR}/mbb_team_names_to_number.csv", index_col=0)
df

,team_name,team_id
0,A&M-Corpus Christi,560639
1,Abilene Christian,560649
2,Air Force,561195
3,Akron,560651
4,Alabama,560658
...,...,...
357,Wright St.,561255
358,Wyoming,561258
359,Xavier,561259
360,Yale,561262


In [139]:
from mbp.constants import DAYS, OFF_SEASON_START, REGULAR_SEASON_START

# https://stats.ncaa.org/contests/livestream_scoreboards?utf8=%E2%9C%93&game_sport_year_ctl_id=16460&conference_id=0&conference_id=0&tournament_id=&division=11&commit=Submit

SEASON = ""
TODAY = datetime.now(timezone("EST"))
LASTWEEK = (TODAY - timedelta(days=DAYS))
DATE_TO = TODAY.strftime("%m/%d/%y")
DATE_FROM = LASTWEEK.strftime("%m/%d/%y")

CURRENT_MONTH = TODAY.strftime("%m")
print(f"Current month: {CURRENT_MONTH}")

all_season_types = pd.DataFrame()

Current month: 10


In [144]:
team_id = df[:][df['team_name'].str.contains("Washington St.")]['team_id'].iloc[0]
team_id

'561226'

In [240]:
raw_year = '2023'
dt = datetime.strptime(raw_year, '%Y')
formatted_date = f"{raw_year}-{str(int(dt.strftime('%y')) + 1)}"
formatted_date

'2023-24'

In [214]:
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By

select_form = driver.find_element(By.ID, "year_list")
select = Select(select_form.find_element(By.NAME, 'year_id'))

# Format date
dt = datetime.strptime(raw_year, '%Y')
formatted_date = f"{raw_year}-{str(int(dt.strftime('%y')) + 1)}"

select.select_by_value(formatted_date)

NoSuchElementException: Message: Unable to locate element: [name="year_id"]; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16


In [219]:
from mbp.webscraping import TEAMS_URL
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By

team_name = "Washington St."
team_id = df[:][df['team_name'].str.contains(team_name)]['team_id'].iloc[0]

team_url = f"{TEAMS_URL}/{team_id}"
wait = WebDriverWait(driver, 10)
wait.until(lambda d: driver.find_element(By.CSS_SELECTOR, "a[target=ATHLETICS_URL]"))

select_form = driver.find_element(By.ID, "year_list")
select = Select(select_form)

# Format date
dt = datetime.strptime(raw_year, '%Y')
formatted_date = f"{raw_year}-{str(int(dt.strftime('%y')) + 1)}"

select.select_by_visible_text(formatted_date)

In [223]:
driver.get(team_url)


In [198]:
from mbp.webscraping import TEAMS_URL
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By

team_name = "Washington St."
team_id = df[:][df['team_name'].str.contains(team_name)]['team_id'].iloc[0]

team_url = f"{TEAMS_URL}/{team_id}"
wait = WebDriverWait(driver, 10)
wait.until(lambda d: driver.find_element(By.CSS_SELECTOR, "a[target=ATHLETICS_URL]"))

# Select the correct year
select = driver.find_element(By.ID, "year_list")

# Format date
dt = datetime.strptime(raw_year, '%Y')
formatted_date = f"{raw_year}-{str(int(dt.strftime('%y')) + 1)}"

select.select_by_value(formatted_date)

tables = driver.find_elements(By.CSS_SELECTOR, "#contentarea table>tbody")

schedule_result_element = None
for table in tables:
    try:
        legend = table.find_element(By.CSS_SELECTOR, "fieldset>legend")
        print(legend.text)
        if legend.text == "Schedule/Results":
            schedule_result_element = table
    except:
        pass

Schedule/Results


In [199]:
# Find the game dates, opponent, results, and attendance (if available)
listing = schedule_result_element.find_elements(By.CSS_SELECTOR, "tbody > tr")
games = []
for li in listing:
    style = li.get_attribute("style")
    if li.get_attribute("style") == "":
        games.append(li)
len(games)

32

In [200]:
season_games = []
for (idx, game) in enumerate(games[1:]):
    # for i in range(4):
    rows = game.find_elements(By.CSS_SELECTOR, "td")
    date = rows[0].text
    opponent = rows[1].text
    try:
        result = rows[2].text or None
    except:
        result = None
    try:
        attendance = rows[3].text or None
    except:
        attendance = None

    res = (date, opponent, result, attendance)
    season_games.append(res)

season_games


[('11/06/2023 TBA', 'Idaho', None, None),
 ('11/10/2023 TBA', 'Prairie View', None, None),
 ('11/18/2023 TBA', 'Mississippi St.\n@Uncasville, CT', None, None),
 ('11/19/2023 TBA', 'TBA\n@Uncasville, CT', None, None),
 ('11/24/2023 TBA', 'Utah Tech', None, None),
 ('11/27/2023 TBA', 'Eastern Wash.', None, None),
 ('12/02/2023 TBA', 'Portland St.', None, None),
 ('12/06/2023 TBA', 'UC Riverside', None, None),
 ('12/10/2023 TBA', 'Grambling', None, None),
 ('12/16/2023 TBA', 'Santa Clara\n@Phoenix, AZ', None, None),
 ('12/23/2023 TBA', 'Boise St.\n@Spokane, WA', None, None),
 ('12/29/2023 TBA', '@ Colorado', None, None),
 ('12/31/2023 TBA', '@ Utah', None, None),
 ('01/04/2024 TBA', 'Oregon St.', None, None),
 ('01/06/2024 TBA', 'Oregon', None, None),
 ('01/10/2024 10:30 PM', '@ Southern California', None, None),
 ('01/13/2024 TBA', 'Arizona', None, None),
 ('01/18/2024 11:00 PM', '@ Stanford', None, None),
 ('01/20/2024 05:00 PM', '@ California', None, None),
 ('01/24/2024 TBA', 'Utah', 

In [201]:
games_df = pd.DataFrame(season_games)
games_df.rename(columns={0: 'raw_datetime', 1: 'opponent', 2: 'result', 3: 'attendance'}, inplace=True)
games_df

,raw_datetime,opponent,result,attendance
0,11/06/2023 TBA,Idaho,None,None
1,11/10/2023 TBA,Prairie View,None,None
2,11/18/2023 TBA,"Mississippi St.\n@Uncasville, CT",None,None
3,11/19/2023 TBA,"TBA\n@Uncasville, CT",None,None
4,11/24/2023 TBA,Utah Tech,None,None
5,11/27/2023 TBA,Eastern Wash.,None,None
6,12/02/2023 TBA,Portland St.,None,None
7,12/06/2023 TBA,UC Riverside,None,None
8,12/10/2023 TBA,Grambling,None,None
9,12/16/2023 TBA,"Santa Clara\n@Phoenix, AZ",None,None


In [203]:
def split_and_parse(s: str):
    parts = s.split(' ')
    dt = datetime.strptime(parts[0], '%m/%d/%Y')

    if parts[1] == 'TBA':
        time = None
    else:
        time_str = ' '.join(parts[1:])
        time = datetime.strptime(time_str, '%I:%M %p')
        dt = dt.replace(hour=time.hour, minute=time.minute)

    return dt

games_df2 = games_df.copy()
for idx, row in games_df.iterrows():
    dt = split_and_parse(row['raw_datetime'])
    games_df2.at[idx, 'datetime'] = pd.to_datetime(dt)

games_df3 = games_df2.drop(columns='raw_datetime')
games_df3

# Year
year = '2023'

save_dir = Path(f"{RAW_DATA_DIR}") / "seasons" / year
if not save_dir.exists():
    save_dir.mkdir(parents=True, exist_ok=True)

games_df3.to_csv(save_dir / f"{team_name}_{year}.csv")

In [234]:
from mbp.webscraping import get_team_games_for_year
from mbp.paths import SEASONS_DIR

team_name = 'Arizona'
year = '2023'

games_df3 = get_team_games_for_year(driver, df, team_name, year)
save_path = SEASONS_DIR / year / f"{team_name}.csv"
games_df3.to_csv(save_path)

In [243]:
from mbp.data import download_raw_data_for_team

# washst_df = download_raw_data_for_team('Washington St.', '2023')
arizona_df = download_raw_data_for_team('Arizona', '2023')

arizona_df.head()

Installing firefox...
executable_path: /Users/auser/.local/bin/geckodriver


,opponent,result,attendance,datetime
0,Morgan St.,None,None,2023-11-06 21:30:00
1,@ Duke,None,None,2023-11-10 19:00:00
2,Southern U.,None,None,2023-11-13 20:00:00
3,Belmont,None,None,2023-11-17 23:00:00
4,UT Arlington,None,None,2023-11-19 18:00:00
